# Experiment 4: Isolate and Classify Wetland only Pixels
- New and Balanced dataset has needed to be created for this experiment.
- The dataset is created by using the following classes:
    - Wetland only training data
 
Want to mask out any pixels in the stack that are note classified as wetland.
The resulting image will be the one that the rf model is applied to.

what we needs to be done before doing the experiment:
- need the resulting classification from dtm classification from experiment 3
- need to find out how to create a mask from the classification

Workflow:
- going to follow the same workflow as the previous experiments
- the only difference is that the mask will be applied to the image stack before the rf model is used for prediction
- something to try: mask the stack from the very beginning and see if the model can predict the wetland pixels

Products that need to be Exported:
- Wetland only features
- Classification of Wetland
- error matrix, accuracy, producer's accuracy, user's accuracy

```python
land_cover_colors = {
    "Bog": "#8B4513",          # Brown
    "Fen": "#808000",          # Olive Green
    "Marsh": "#008000",        # Medium Green
    "Salt marsh": "#9ACD32",   # Yellow-Green
    "Shallow water": "#ADD8E6",# Light Blue
    "Swamp": "#006400"         # Dark Green
}
```

cloud folder
nblidar/experiments/exp4/{elevation}/prediction/**.tif
nblidar/experiments/exp4/{elevation}/error_matrix.geojson 


In [1]:
import ee
import geemap



In [2]:
ee.Initialize(project='nb-lidar')

In [3]:
# DEFINE CONSTANTS
folder = "exp4"
aoi_id = "projects/nb-lidar/assets/aoi_nb_south"
input_data_id = "projects/nb-lidar/assets/training_data/wetland_only"
terrain_type = "dsm"

In [4]:
aoi = ee.FeatureCollection(aoi_id).geometry()
input_data = ee.FeatureCollection(input_data_id)

In [5]:
from src import image_processing as proc

s1 = proc.fetch_and_proecss_s1_seasonal(aoi)
s2 = proc.fetch_and_proeces_s2_sr_seasonal(aoi)
al = proc.fetch_and_process_alos()
ft = proc.fetch_fourier_transform()
ta = proc.fetch_terrain(terrain_type)

stack = ee.Image.cat(s1, s2, al, ft, ta)

In [6]:
# create a wetland mask
wetland_mask = ee.Image('projects/nb-lidar/assets/exp3_dtm_classification').eq(3) # 3 = wetland class value

In [7]:
# apply mask to the stack
masked_stack = stack.updateMask(wetland_mask)

In [8]:
# Extract Features
features = masked_stack.sampleRegions(
    collection=input_data,
    tileScale=16,
    scale=10
)

In [9]:
# split features into train and test
train = features.filter('random < 0.7')
test = features.filter('random >= 0.7')

In [10]:
# Train the Random Forest Model
rf = ee.classifier.Classifier.smileRandomForest(1000).train(train, 'class_name', masked_stack.bandNames())

In [11]:
# assess the model
# assessment
order = test.aggregate_array('class_name').distinct()
validated = test.classify(rf)
error_matrix = validated.errorMatrix("class_name", "classification", order=order)

In [12]:
error_matrix

In [13]:
error_matrix.accuracy().multiply(100).format("%.2f")

In [14]:
print("Conusmers Accuracy")
error_matrix.consumersAccuracy().toList().flatten().map(lambda x: ee.Number(x).multiply(100).format("%.2f"))

Conusmers Accuracy


In [15]:
print("Producers Accuracy")
error_matrix.producersAccuracy().toList().flatten().map(lambda x: ee.Number(x).multiply(100).format("%.2f"))

Producers Accuracy


In [16]:
# make prediction using the model
prediction = masked_stack.classify(rf)

In [17]:
land_cover_colors = {
    "Bog": "#8B4513",          # Brown
    "Fen": "#808000",          # Olive Green
    "Marsh": "#008000",        # Medium Green
    "Salt marsh": "#9ACD32",   # Yellow-Green
    "Shallow water": "#ADD8E6",# Light Blue
    "Swamp": "#006400"         # Dark Green
}

In [18]:
Map = geemap.Map()

Map.addLayer(prediction, {'palette': list(land_cover_colors.values()), 'min': 1, 'max': 6}, 'Prediction')
Map.addLayerControl()
Map.centerObject(aoi, zoom=8)

Map

Map(center=[45.74809742262684, -65.99999969409251], controls=(WidgetControl(options=['position', 'transparent_…

In [19]:
from src.models.train_and_assess import format_error_matrix_for_export


fmt_error_matrix = format_error_matrix_for_export(error_matrix)
fmt_error_matrix